<a href="https://colab.research.google.com/github/vatsasree/Reading-Order-using-Language-Properties/blob/nikhil_testing/doctTr_Reading_Order_English.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install "python-doctr[torch]==0.5.1"
!pip install rapidfuzz==2.6.0 #dependency
!pip install tensorflow_addons
!pip install tf2onnx
# !pip install --upgrade pillow==6.2.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.2/205.2 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 9.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 9.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 kB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 93.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.1/75.1 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 5.6 MB/s eta 0:00:00
  Attempting uninstall: rapidfuzz
    Found existing installation: rapidfuzz 3.6.1
    Uninstalling rapidfuzz-3.6.1:
      Successfully uninstalled rapidfuzz-3.6.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.8/455.8 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 57.6 MB/s eta 0:00:00


In [2]:
import os
os.environ['USE_TORCH'] = '1'

In [3]:
import os
import torch
import os
from doctr.models import ocr_predictor
from collections import OrderedDict
from doctr.io import DocumentFile
import cv2
import matplotlib.pyplot as plt

In [4]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


NOTE: MODEL FILE NOT REQUIRED FOR TEXT RECOGNITION INFERENCE

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_file = '/content/drive/MyDrive/Reading-Order-using-Language-Properties-main/weights/db_resnet50.pt'

if isinstance(model_file, str):
    predictor = ocr_predictor(pretrained=True)
    state_dict = torch.load(model_file)
    # print(state_dict)
    new_state_dict = OrderedDict()
    for k, v in state_dict.items():
        name = k[7:] # remove `module.`
        new_state_dict[name] = v
        # print(k)
    model = predictor.det_predictor.model
    print(type(model))
    print(model)
    # model.load_state_dict(new_state_dict)
    predictor.det_predictor.model.load_state_dict(new_state_dict)
else:
    predictor = ocr_predictor(pretrained=True)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


  0%|          | 0/101971449 [00:00<?, ?it/s]

  0%|          | 0/63286381 [00:00<?, ?it/s]

<class 'doctr.models.detection.differentiable_binarization.pytorch.DBNet'>
DBNet(
  (feat_extractor): IntermediateLayerGetter(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_ru

In [6]:
doc = DocumentFile.from_images('/content/drive/MyDrive/Reading-Order-using-Language-Properties-main/Examples/readingorder1column_03.png')
result = predictor(doc)
dic = result.export()

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


In [7]:
dic

{'pages': [{'page_idx': 0,
   'dimensions': (826, 692),
   'orientation': {'value': None, 'confidence': None},
   'language': {'value': None, 'confidence': None},
   'blocks': [{'geometry': ((0.001953125, 0.0107421875),
      (0.990234375, 0.5751953125)),
     'lines': [{'geometry': ((0.076171875, 0.0107421875),
        (0.986328125, 0.046875)),
       'words': [{'value': 'In',
         'confidence': 0.9999947547912598,
         'geometry': ((0.076171875, 0.0107421875),
          (0.1162109375, 0.0400390625))},
        {'value': 'Figure',
         'confidence': 0.9355168342590332,
         'geometry': ((0.125, 0.01171875), (0.240234375, 0.046875))},
        {'value': '2',
         'confidence': 0.999958872795105,
         'geometry': ((0.2490234375, 0.0126953125), (0.26953125, 0.0390625))},
        {'value': 'we',
         'confidence': 0.9997571110725403,
         'geometry': ((0.279296875, 0.0205078125), (0.326171875, 0.0390625))},
        {'value': 'show',
         'confidence': 0.9

In [8]:
def doctr_predictions(dic):



    page_dims = [page['dimensions'] for page in dic['pages']]

    regions = []
    abs_coords = []

    regions = [[word for block in page['blocks'] for line in block['lines'] for word in line['words']] for page in dic['pages']]
    abs_coords = [
    [[int(round(word['geometry'][0][0] * dims[1])),
      int(round(word['geometry'][0][1] * dims[0])),
      int(round(word['geometry'][1][0] * dims[1])),
      int(round(word['geometry'][1][1] * dims[0]))] for word in words]
    for words, dims in zip(regions, page_dims)
    ]

    return abs_coords

In [9]:
doctr_predictions(dic)

[[[53, 9, 80, 33],
  [86, 10, 166, 39],
  [172, 10, 187, 32],
  [193, 17, 226, 32],
  [232, 10, 292, 31],
  [299, 10, 338, 32],
  [345, 14, 431, 37],
  [439, 9, 465, 33],
  [469, 12, 547, 32],
  [556, 17, 606, 38],
  [612, 10, 647, 32],
  [653, 9, 683, 33],
  [4, 39, 67, 62],
  [76, 40, 166, 64],
  [175, 39, 201, 62],
  [208, 39, 247, 60],
  [255, 38, 312, 61],
  [320, 40, 406, 67],
  [415, 40, 489, 60],
  [502, 40, 544, 61],
  [553, 39, 591, 61],
  [601, 40, 651, 61],
  [659, 38, 685, 62],
  [5, 69, 85, 95],
  [95, 68, 134, 90],
  [143, 69, 217, 90],
  [226, 69, 368, 97],
  [376, 69, 426, 90],
  [435, 69, 489, 90],
  [498, 69, 553, 90],
  [562, 70, 629, 94],
  [638, 67, 682, 91],
  [4, 98, 54, 119],
  [64, 98, 208, 119],
  [218, 95, 268, 126],
  [275, 97, 363, 119],
  [372, 98, 437, 119],
  [445, 96, 472, 120],
  [476, 98, 555, 125],
  [564, 95, 588, 127],
  [597, 104, 634, 119],
  [643, 100, 682, 119],
  [3, 127, 74, 156],
  [93, 127, 163, 148],
  [174, 131, 261, 153],
  [272, 136, 3

In [10]:
dic['pages'][0]['dimensions']

(826, 692)

In [11]:
import pandas as pd

def create_dataframe_from_doctr(dic):

    page_dims = [page['dimensions'] for page in dic['pages']]


    data = []


    page_number = 1
    for page, dims in zip(dic['pages'], page_dims):
        block_number = 1
        for block in page['blocks']:
            line_number = 1
            for line in block['lines']:
                for word in line['words']:


                    top_left_x = int(round(word['geometry'][0][0] * dims[1])),
                    top_left_y = int(round(word['geometry'][0][1] * dims[0])),
                    bottom_right_x = int(round(word['geometry'][1][0] * dims[1])),
                    bottom_right_y =int(round(word['geometry'][1][1] * dims[0]))



                    data.append({
                        'Page Number': page_number,
                        'Block Number': block_number,
                        'Line Number': line_number,
                        'Word': word['value'],
                        'Confidence': word['confidence'],
                        'Top-Left X': top_left_x,
                        'Top-Left Y': top_left_y,
                        'Bottom-Right X': bottom_right_x,
                        'Bottom-Right Y': bottom_right_y
                    })
                line_number += 1
            block_number += 1
        page_number += 1


    return pd.DataFrame(data)


df = create_dataframe_from_doctr(dic)


In [12]:
df.sort_values(by=['Top-Left Y', 'Top-Left X'])

,Page Number,Block Number,Line Number,Word,Confidence,Top-Left X,Top-Left Y,Bottom-Right X,Bottom-Right Y
0,1,1,1,In,0.999995,"(53,)","(9,)","(80,)",33
7,1,1,1,of,0.999997,"(439,)","(9,)","(465,)",33
11,1,1,1,all,0.998347,"(653,)","(9,)","(683,)",33
1,1,1,1,Figure,0.935517,"(86,)","(10,)","(166,)",39
2,1,1,1,2,0.999959,"(172,)","(10,)","(187,)",32
...,...,...,...,...,...,...,...,...,...
251,1,4,1,well.,0.984686,"(380,)","(799,)","(434,)",822
247,1,4,1,normalized,0.976810,"(5,)","(800,)","(135,)",820
248,1,4,1,between,0.999947,"(147,)","(800,)","(244,)",820
249,1,4,1,models,0.988787,"(253,)","(800,)","(337,)",820


In [13]:
def split_into_sentences(df):
    sentences = []
    current_sentence = ""

    for _, row in df.iterrows():
        word = row['Word']
        current_sentence += word + " "

        # Check if the word ends with a sentence-ending punctuation
        if any(word.endswith(punct) for punct in ['.', '!', '?']):
            sentences.append(current_sentence.strip())
            current_sentence = ""

    # Add the last sentence if it's not empty
    if current_sentence:
        sentences.append(current_sentence.strip())

    return sentences

# Assuming 'df' is your DataFrame sorted by 'Top-Left Y' and 'Top-Left X'
sentences = split_into_sentences(df)


In [14]:
raw = sentences

In [15]:
import sys, os, re, json
from collections import Counter, OrderedDict
import itertools
from numpy import *
import pandas as pd
# based on http://cs229.stanford.edu/proj2014/Ian%20Tenney,%20A%20General-Purpose%20Sentence-Level%20Nonsense%20Detector.pdf
# and https://github.com/iftenney/nlp-nonsense

In [16]:
def make_basic_features(df):
    """Compute basic features."""

    df['f_nchars'] = df['__TEXT__'].map(len)
    df['f_nwords'] = df['word'].map(len)

    punct_counter = lambda s: sum(1 for c in s
                                  if (not c.isalnum())
                                      and not c in
                                        [" ", "\t"])
    df['f_npunct'] = df['__TEXT__'].map(punct_counter)
    df['f_rpunct'] = df['f_npunct'] / df['f_nchars']

    df['f_ndigit'] = df['__TEXT__'].map(lambda s: sum(1 for c in s
                                  if c.isdigit()))
    df['f_rdigit'] = df['f_ndigit'] / df['f_nchars']

    upper_counter = lambda s: sum(1 for c in s if c.isupper())
    df['f_nupper'] = df['__TEXT__'].map(upper_counter)
    df['f_rupper'] = df['f_nupper'] / df['f_nchars']

    # fraction named entities recognized (ner) -- 'O' is not recognized
    df['f_nner'] = df['ner'].map(lambda ts: sum(1 for t in ts
                                              if t != 'O'))
    df['f_rner'] = df['f_nner'] / df['f_nwords']

    # Check standard sentence pattern:
    # if starts with capital, ends with .?!
    def check_sentence_pattern(s):
        ss = s.strip(r"""`"'""").strip()
        return s[0].isupper() and (s[-1] in '.?!\n')
    df['f_sentence_pattern'] = df['__TEXT__'].map(check_sentence_pattern)

    # Normalize any LM features
    # by dividing logscore by number of words
    lm_cols = {c:re.sub("_lmscore_", "_lmscore_norm_",c)
               for c in df.columns if c.startswith("f_lmscore")}
    for c,cnew in lm_cols.items():
        df[cnew] = df[c] / df['f_nwords']

    return df


In [17]:

import spacy
import en_core_web_sm # or en_core_web_lg if need tokenization.
nlp = en_core_web_sm.load()

In [18]:
data = []
for idx,text in enumerate(raw):
    doc = nlp(text)
    row = {"__TEXT__": text}
    row['ner'] = [i.pos_ for i in doc]
    row['sentiment'] = doc.sentiment
    row['word'] = [i.text for i in doc]
    data.append(row)

df = pd.DataFrame(data)

In [19]:
df = make_basic_features(df)

<ipython-input-16-3d5227a41f43>:7: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  punct_counter = lambda s: sum(1 for c in s
<ipython-input-16-3d5227a41f43>:14: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  df['f_ndigit'] = df['__TEXT__'].map(lambda s: sum(1 for c in s
<ipython-input-16-3d5227a41f43>:18: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  upper_counter = lambda s: sum(1 for c in s if c.isupper())
<ipython-input-16-3d5227a41f43>:23: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or th

In [20]:
"""next: Apply rules
1 Baseline sentence heuristic: first letter is Capitalized, and line ends with one of .?! (1 feature).
2 Number of characters, words, punctuation, digits, and named entities (from Stanford CoreNLP NER tagger), and normalized versions by text length (10 features).
3 Part-of-speech distributional tags: (# / # words) for each Penn treebank tag (45 features).
4 Indicators for the part of speech tag of the first and last token in the text (45x2 = 90 features).
5 Language model raw score (s lm = log p(text)) and normalized score (s¯lm = s lm / # words) (2 features).
"""
cleaned = {}
dirty = {}
bad = 0
for idx,row in df.iterrows():
    if row['f_sentence_pattern'] and (row['f_npunct'] + row['f_nwords']) > 5 and row['f_nner'] > 0:
        cleaned[idx] = row['__TEXT__']
    else:
        dirty[idx] = row['__TEXT__']
        bad += 1
print(bad,len(df))
from pprint import pprint as pp

0 10


In [21]:
pp(cleaned)

{0: 'In Figure 2 we show the spectra of cosmic rays for all three models, in '
    'the local plasma frame.',
 1: 'For the sake of clarity, the cooled populations that have been swept fur- '
    'ther downstream (the interior shells of Figure 1) are not given.',
 2: 'These spectra were taken just downstream from the subshock at A - 0, to '
    'most closely match the spectral indices predicted by analytical work.',
 3: 'Though we assume an 1sotropic distribution of particles for all photon '
    'pro- duction processes, this IS obviously not the case at the subshock.',
 4: 'All particles require several diffusion lengths of transport downstream '
    'before their angular distribu- tion relaxes into isotropy.',
 5: 'Since the diffusion length 1S an increasing function of momentum '
    '(Equation 1) higher- energy particles remain anisotropic for longer '
    'distances downstream.',
 6: 'The effects of anisotropic particle distribu- tions will be addressed in '
    'future work; for o

In [22]:
pp(dirty)

{}
